### 1. Import Library

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### 2.Crawl_data_q_a

#### 2.1.Find Elements

In [2]:
# Hàm để cào dữ liệu từ một trang
def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('div', class_='box-item-content')
    articles_data = []
    
    for _, article in enumerate(articles, start=1):
        # Lấy thông tin tác giả và thời gian
        author_info = article.find('div', class_='box-item-top')
        name = author_info.find('span', class_='author').get_text(strip=True)
        time = author_info.find('span', class_='time').get_text(strip=True)

        # Lấy tiêu đề câu hỏi
        question_element = article.find('a', class_='question-title')
        question = question_element.get_text(strip=True)
        
        # Lấy tình huống
        summary = article.find('div', class_='question-sapo').get_text(strip=True)
        
        # Lấy liên kết chi tiết
        detail_link = article.find('a', class_='box-viewmore')['href']
        detail_url = f"https://chinhsachonline.chinhphu.vn{detail_link}"
        
        # Gửi yêu cầu HTTP để lấy nội dung trang chi tiết
        detail_response = requests.get(detail_url)
        detail_soup = BeautifulSoup(detail_response.content, 'html.parser')
        
        # Lấy câu trả lời từ trang chi tiết
        answer_div = detail_soup.find('div', class_='detail__rcontent')
        answer = answer_div.get_text(strip=True) if answer_div else 'No answer available'
        
        # Thêm thông tin bài viết vào danh sách
        articles_data.append({
            'Name': name,
            'Time': time,
            'Question': question,
            'Situation': summary,
            'Answer': answer
        })
    
    return articles_data


#### 2.2.Execute Function

In [3]:
articles_data = []
for page_num in range(1, 400):  # Thay đổi phạm vi để bao gồm số trang bạn cần
    
    page_url = f"https://chinhsachonline.chinhphu.vn/chinh-sach-voi-nguoi-co-cong/t1/2/trang-{page_num}.htm"
    articles_data.extend(crawl_page(page_url))

#### 2.3 Export csv

In [4]:
# Tạo DataFrame từ danh sách dữ liệu
df_chinhsach = pd.DataFrame(articles_data)
df_chinhsach.head(5)

,Name,Time,Question,Situation,Answer
0,Nguyễn Việt Quang,08:30 28/03/2025,Căn cứ xác nhận người có công giúp đỡ cách mạng,"Tôi xin hỏi, thế nào được coi là người trong g...",Bộ Lao động – Thương binh và Xã hội (nay là Bộ...
1,Lê Kim Miên,09:41 18/03/2025,Chế độ ưu đãi đối với thân nhân liệt sĩ,Tôi đang hưởng chế độ trợ cấp vợ liệt sĩ tái g...,Bộ Lao động - Thương binh và Xã hội (nay là Bộ...
2,Nguyễn Anh Hào,11:05 11/03/2025,"Có được hưởng nhiều lần chính sách miễn, giảm ...","Mẹ tôi là thương binh, tỷ lệ 4/4, có Huân chươ...",Bộ Lao động – Thương binh và Xã hội (cũ) trả l...
3,Nguyễn Trang,14:05 11/03/2025,Thủ tục chấm dứt ủy quyền hưởng chế độ thờ cún...,Đề nghị cơ quan chức năng giải đáp trường hợp ...,Bộ Lao động – Thương binh và Xã hội (cũ) trả l...
4,Nguyễn Thị Hưởng,08:28 07/03/2025,Có thể đính chính tên liệt sĩ tại nghĩa trang ...,"Bố tôi hy sinh tại chiến trường Miền Nam, an t...",Sở Lao động – Thương binh và Xã hội tỉnh Tây N...


In [5]:

# Lưu DataFrame vào file CSV
csv_file = '../Source_Fine-Tuning/data/chinhsach_nguoicocong_data.csv'
df_chinhsach.to_csv(csv_file, encoding='utf-8',index_label='Index')

print(f"Successfully saved data to {csv_file}")

Successfully saved data to ../Source_Fine-Tuning/data/chinhsach_nguoicocong_data.csv


In [6]:
df_chinhsach=pd.read_csv("../Source_Fine-Tuning/data/chinhsach_nguoicocong_data.csv",encoding='utf-8')

In [7]:
df_chinhsach.shape

(784, 6)